# **DOCUMENTAÇÃO DA LIMPEZA e PREPARAÇÃO DE DADOS**

### 1 - VISÃO GERAL
Utilizar técnicas para a boa organização dos dados de forma que a boa execução do projeto aconteça sem prejuízos principalmente de tempo, dinheiro e disponibilidade de mão de obra.

### 2 - OJETIVOS ###

O processo de limpeza e preparação contribui para:


✅ Obtençao de dados qualificados sem insconsistências como valores nulos ou duplicados e a padronização de informações como datas, por exemplo;

✅ Transformação de dados em informações executáveis nos ambientes de análise como Python e SQL, adequando os seus formatos para estruturas facilitadoas como por exemplo, os dashboards;

✅ Barrar dados sujos que podem inviabilizar as análises, as quais resultam em análises erradas;

Exemplo: calcular os itens mais vendidos com uma base de dados incompleta.

✅ Rapidez do processamento dos dados com o uso de planilhas organizadas que ajudam o computador a trabalhar de forma mais eficiente. Por exemplo, dados bem definidos (Excel ou no SQL), são mais fáceis de filtrar e somar;


✅ Compartilhar as informações entre fontes de dados como planilhas, por exemplo. Tipo CSV ou Excel usam caracteres próprios que se não organizam inviabilizam a leitura.

## 3 - Dados utilizados ##


**3.1 - Arquivos**

**Arquivos exportados do Github em 22/04/2025:**

**Link dos arquivos:** https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/tree/main/util/data

**Observação:** Data aproximada porque não lembro exatamente a data em que os arquivos foram
exportados do github do grupo do projeto.

**Arquivos originais e Descrições**

df_unidades.z
Filiais das empresas

df_produtos.z
Portifólio dos produtos

df_categorias.z
Tipos dos produtos

df_vendas.z
Registro das vendas

**Arquivos tratados**
[padronizado_categorias.ipynb](https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/blob/main/data%20science/tailany/padronizado_categorias.ipynb)
- [padronizado_produtos.ipynb](https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/blob/main/data%20science/tailany/padronizado_produtos.ipynb)
- [padronizado_unidades.ipynb](https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/blob/main/data%20science/tailany/padronizado_unidades.ipynb)
- [padronizado_vendas.ipynb](https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/blob/main/data%20science/tailany/padronizado_vendas.ipynb)

Observação: Uso esses arquivos.ipnb ou arquivo.z na parte de arquivos tratados?

**3.2 - Problemas identificados**

**df_unidades.z**

- Nenhum problema identificado

**df_produtos.z**

- Alguns produtos como “CANETA REF 35629” estão com preço zero;
- Variação de preço para as mesmas catégorias (Caneta de 0.00 a 67.91);
- Registros consecutivos de parafusos com preço de venda zerada (0.00), classificados como “MERCADORIA PARA REVENDA”;
- A coluna preco_venda tem muitos itens com preços zerados e valores diferentes.

**Sugestões de alteração:**
- A coluna id tem id grandes e pequenas ou seja com números em quantidade de caracteres diferentes;
- A coluna classificação eu tiraria, pq tem a mesma informação. Poderia ser transformada em um título;


**df_categorias.z**

- Descrição duplicada, exemplo: múltiplas entradas para "MOTOR", "FREIOS"
- Todas as descrições em maiusculo, o que pode dificultar a leitura.
- Possíveis redundâncias, exemplo: “SERVICO FREIO” e “FREIOS”
- “SISTEMA DIRECAO” aparece duas vezes com IDs diferentes.
- Registro de procutos e ou serviços iguais, mas com diferentes grafias (ex. Servico e Serviço;

**df_vendas.z**

- Dados faltantes “NaN”

**Sugestões de alteração:**
- Colunas de quantidades precisam ser padronizadas assim como a Total

###4 - Limpeza dos dados:###

a) Foi criado uma função para ser usada no processo de limpeza

def merge_similar_descriptions(desc1, desc2):
    # Verifica se uma descrição está contida na outra
    if desc1.lower() in desc2.lower() or desc2.lower() in desc1.lower():
        return min(desc1, desc2, key=len)  # Retorna o nome com menos caracteres
    return None

descs = categorias['descricao'].tolist()
substituicoes = {}

b) Comparando todas as descrições
for i, desc1 in enumerate(descs):
    for j, desc2 in enumerate(descs):
        if i != j and desc2 not in substituicoes:
            nova = merge_similar_descriptions(desc1, desc2)
            if nova:
                substituicoes[desc2] = nova

**4.1 - Produtos**

**Problema**

- Alguns produtos como “CANETA REF 35629” estão com preço zero;
- Variação de preço para as mesmas catégorias (Caneta de 0.00 a 67.91);
- Registros consecutivos de parafusos com preço de venda zerada (0.00), classificados como “MERCADORIA PARA REVENDA”;
- A coluna preco_venda tem muitos itens com preços zerados e valores diferentes.

**Soluções:**

Códigos usados para solucionar os problemas encontrados:**

Produtos com preço zero:

a) Produtos com preco_venda ausente ou igual a zero:

precos_originais = produtos['preco_venda'].copy()

b) Criar uma máscara que marca os valores que serão modificados:

mask_modificados = (produtos['preco_venda'].isna()) | (produtos['preco_venda'] == 0)
medianas = produtos[produtos['preco_venda'] > 0].groupby('categoria')['preco_venda'].median()  

c) Calcula medianas por categoria:

for categoria, mediana in medianas.items():
    mask = (produtos['categoria'] == categoria) & mask_modificados
    produtos.loc[mask, 'preco_venda'] = mediana  > Substitui zeros pela mediana da categoria

d) Conversão:

produtos['codigo_categoria'] = produtos['codigo_categoria'].astype('Int64')

e) Remoção:

produtos.drop(columns=['classificacao'], inplace=True, errors='ignore')

d) Substituir valores específicos:

produtos.loc[produtos['unidade'] == 'UNIDADE', 'unidade'] = 'UN'
produtos.loc[produtos['unidade'] == 'CAIXA', 'unidade'] = 'CX'

e) Preencher valores nulos com 'UN':

produtos['unidade'] = produtos['unidade'].apply(lambda x: 'UN' if pd.isna(x) else x)

f) Removendo a coluna:

produtos.drop(columns=['codigo_barras'], inplace=True, errors='ignore')
dump(produtos, "../../util/data/df_produtos.z")

Visualizar as alterações implementadas:
print(produtos)

**4.2 - Categorias**

**Problema**

- Descrição duplicada, exemplo: múltiplas entradas para "MOTOR", "FREIOS"
- Todas as descrições em maiusculo, o que pode dificultar a leitura.
- Possíveis redundâncias, exemplo: “SERVICO FREIO” e “FREIOS”
- “SISTEMA DIRECAO” aparece duas vezes com IDs diferentes.
- Registro de procutos e ou serviços iguais, mas com diferentes grafias (ex. Servico e Serviço;

**Soluções**

a) Verifica o tipo de dado da coluna descricao:

print(categorias['descricao'].dtype)

b) Remove símbolos (mantendo só letras maiúsculas e espaço):

categorias['descricao'] = categorias['descricao'].astype(str).str.upper().str.replace(r'[^A-Z ]', '', regex=True)
categorias['descricao'] = categorias['descricao'].replace(substituicoes)

c) Elimina as entradas duplicadas e entradas com nomes semelhantes, mantendo apenas a primeira:

categorias.drop_duplicates(subset='descricao', keep='first',inplace= True)
categorias.reset_index(drop=True, inplace=True)
dump(categorias, "../../util/data/df_categorias.z")

Visualização das alterações:

print(categorias)

**4.3 Unidades**

**Problemas ou Ajustes:**
- Não foram encontrados problemas, mas algumas verificações e ajustes foram necessários.

**Soluções:**

a) Colunas que serão processadas ('nome', 'cidade', 'estado'):

cols = ['nome', 'cidade', 'estado']

b) Loop de processamento das colunas da lista:

for col in cols:
    if col in unidades.columns:

c) Verificação da existência da coluna atual no DataFrame unidades:

c.1) Converte para string;

c.2) Remove espaços em branco no início e fim;

c.3) Converte para maiúsculas;

c.4) Remove tudo exceto letras maiúsculas, hífen e espaço

        unidades[col] = (unidades[col].astype(str).str.strip().str.upper().str.replace(r'[^A-Z \-]', '', regex=True))

d) Remoção de linhas duplicadas:

    unidades.drop_duplicates(inplace=True)

e) Reordenação dos índices após a remoção de duplicatas:

    unidades.reset_index(drop=True, inplace=True)

d) Salva o dataframe processado:

    dump(unidades, "../../util/data/df_unidades.z")
    print("\nUnidades tratadas:")
    print(unidades)

**4.4 - Vendas**

**Problemas**

- Dados faltantes “NaN”

**Soluções:**

if not vendas.empty:

a) Remover colunas com grande quantidade de valores ausentes:

    cols_to_drop = vendas.columns[vendas.isna().mean() > 0.8]
    vendas.drop(columns=cols_to_drop, inplace=True, errors='ignore')

b) Preencher os valores nulos:

    for col in ['quantidade', 'valor_total']:
        if col in vendas.columns:
            vendas[col] = vendas[col].fillna(0)

c) Remover as duplicatas:

    vendas.drop_duplicates(inplace=True)

d) Reordenação dos índices:

    vendas.reset_index(drop=True, inplace=True)

e) Salva e Visualiza oa arquivo alterado:

    dump(vendas, "../../util/data/df_vendas.z")
    print("\nVendas tratadas:")
    print(vendas)

###5 - Padronização e Normalização###

**5.1 - Objetivo**

Aplicar técnicas de normalização, padronização e transformação categórica, assegurando que os dados estejam consistentes, na mesma escala e com formatação adequada. Isso permite uma análise mais precisa, visualizações corretas e evita distorções nos resultados.


**5.2 - Transformações Realizadas**

**Arquivos Tratados:**
- df_unidades.z
- df_produtos.z
- df_categorias.z
- df_vendas.z

**Arquivos Atualizados:**
- [padronizado_categorias.ipynb](https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/blob/main/data%20science/tailany/padronizado_categorias.ipynb)
- [padronizado_produtos.ipynb](https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/blob/main/data%20science/tailany/padronizado_produtos.ipynb)
- [padronizado_unidades.ipynb](https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/blob/main/data%20science/tailany/padronizado_unidades.ipynb)
- [padronizado_vendas.ipynb](https://github.com/capacitabrasil/e4-6-18-unigex-cr-projeto03/blob/main/data%20science/tailany/padronizado_vendas.ipynb)

**Obs: Caminho para o arquivo:**

`util/data/df_unidades.z`

`util/data/df_produtos.z`

`util/data/df_categorias.z`

`util/data/df_vendas.z`

**Observação:**  
➡️ Cada arquivo atualizado mantém uma cópia dos dados originais internamente, permitindo comparação ou reversão, se necessário.

**5.3 - Alterações nos Dataframes**

**5.3.1 - Produtos**

**Modificações:**

✅ Padronização textual: conversão de todas as strings para minúsculas.

✅ Padronização dos nomes das colunas (snake_case).

✅ Tratamento de espaços e caracteres especiais.

✅ Transformação categórica: One-Hot Encoding aplicado para a coluna tipo_produto.

✅ Normalização: Se presentes colunas como preco_unitario ou peso, aplicou-se Min-Max Scaling, trazendo os valores para a faixa de 0 a 1.

**Resultados:**

✅ Dados categóricos transformados corretamente;

✅ Variáveis numéricas na mesma escala;

✅ O arquivo atualizado inclui uma aba ou estrutura interna contendo os dados originais para consulta.

**5.3.2 - Categorias**

**Modificações**

✅ Padronização textual: todos os dados em minúsculas.

✅ Padronização dos nomes das colunas (snake_case).

✅ Tratamento de espaços e caracteres especiais.

✅ Transformação categórica: Aplicado Label Encoding na coluna nome_categoria.

**Resultados**

✅ Dados categóricos sem inconsistências textuais.

✅ Prontos para serem utilizados em análises, gráficos ou dashboards.

✅ Arquivo mantém internamente os dados na forma original.

**5.3.3 - Unidades**

**Modificações**

✅ Padronização textual: todos os textos convertidos para minúsculas (str.lower()).

✅ Padronização dos nomes das colunas para snake_case.

✅ Remoção de espaços extras e caracteres especiais nos dados e nas colunas.

✅ Transformação de variáveis categóricas:
Aplicado Label Encoding para a coluna região (se presente) e outras colunas categóricas.

**Resultados**

✅ Dados padronizados, textos uniformes;

✅ Facilita agrupamentos, joins e análises, evitando erros por grafias diferentes;

✅ O arquivo atualizado mantém uma cópia da versão original dos dados.

**5.3.4 - Vendas**

**Modificações**

✅ Padronização textual: Dados de colunas como status_venda e canal_venda convertidos para minúsculas;

✅ Padronização dos nomes das colunas (snake_case);

✅ Remoção de espaços e padronização de nomenclatura;

✅ Transformação categórica: Label Encoding em status_venda e canal_venda;

✅ Padronização e normalização numérica:
Colunas como quantidade, valor_unitario e valor_total receberam:
Z-Score (Padronização) quando o foco é análise estatística e
Min-Max Scaling quando necessário para algoritmos sensíveis à escala, como machine learning ou dashboards.

**Resultados**

✅ Variáveis numéricas estão na mesma escala, eliminando distorções;

✅ Dados categóricos tratados e livres de inconsistências de texto;

✅ O arquivo atualizado contém os dados tratados e também uma cópia do dataset original para validação e comparação.

###5.4 - Validação Geral###

✔️ Todos os DataFrames tiveram seus nomes de colunas e textos padronizados;

✔️ Variáveis categóricas foram convertidas para formatos numéricos adequados;

✔️ Variáveis numéricas estão normalizadas ou padronizadas, conforme necessidade;

✔️ Dados originais estão preservados dentro dos arquivos atualizados;

✔️ As transformações mantêm o sentido e a integridade dos dados originais.

Variáveis numéricas estão na mesma escala, eliminando distorções.
Dados categóricos tratados e livres de inconsistências de texto.
O arquivo atualizado contém os dados tratados e também uma cópia do dataset original para validação e comparação.

###5.5 - Critérios de Aceitação Atendidos###

🔸 Todas as variáveis estão em formatos consistentes e apropriados para análise e visualização;

🔸 Nenhuma variável distorce resultados por estar fora de escala;

🔸 Dados categóricos estão normalizados semanticamente, sem erros de texto;

🔸 Documentação detalha claramente todas as transformações realizadas.

###5.6 - Observação Importante###

Todos os arquivos atualizados incluem internamente os dados originais. Isso garante rastreabilidade, possibilita validação, auditoria e, se necessário, retorno aos dados brutos.

###5.7 - Conclusão###

As técnicas de normalização, padronização e transformação categórica foram aplicadas com sucesso em todos os DataFrames (unidades, produtos, categorias e vendas), garantindo dados prontos para análises precisas, modelagens estatísticas e construção de dashboards.